# Data Combine

Combine .csv files of the simulation galaxy cluster data and save the data as dataframe(.parque).
This file includes galaxy positions(x, y), projected radius(R_proj), velocity(v_los), flag(cluster membership),
projected axis(proj_axis), galaxy id(gal_id), cluster id(cluster_id).

For the flag, interloper (flag = 0 ), cluster member (if not flag =0). 
Members are all halos in the 5 Mpc sphere of the cluster.
Flag =1 subhalo, Flag = 2 backsplashing, Flag = 3 the other.

# 1. Data load

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

# gradient boosting for regression in scikit-learn
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.datasets import make_classification
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
from lightgbm import LGBMClassifier

/Users/eunbinkim/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
fo = open("/Users/eunbinkim/work/pywork/data/flist.txt","r")
lines = fo.readlines()

In [4]:
lines[0]

'cluster_0001_xy.csv\n'

In [5]:
fname ="./data/"+lines[0].split('.csv')[0]+".csv"

In [6]:
fname

'./data/cluster_0001_xy.csv'

In [7]:
lines[0]

'cluster_0001_xy.csv\n'

In [8]:
cluster_id = lines[0].split('_')[1]
proj_ax = lines[0].split('.')[0].split('_')[2]

In [9]:
fdf = pd.read_csv(fname,skiprows=1)

# 2. Add new columns

In [10]:
fdf['proj_axis'] = proj_ax
fdf['cluster_id'] = cluster_id
fdf['gal_id'] = fdf.index

In [11]:
fdf.head()

,x,y,R_proj,v_los,flag,proj_axis,cluster_id,gal_id
0,1.435010,-4.35463,4.58499,-1727.40,0,xy,0001,0
1,1.076120,-4.05722,4.19751,-1850.02,0,xy,0001,1
2,-4.315750,2.40710,4.94164,-2867.62,0,xy,0001,2
3,0.717368,-4.49094,4.54788,-1746.10,0,xy,0001,3
4,0.198263,-4.51754,4.52189,-1658.32,0,xy,0001,4


In [12]:
fdf.tail()

,x,y,R_proj,v_los,flag,proj_axis,cluster_id,gal_id
300,-0.943515,1.82415,2.05371,3095.18,0,xy,0001,300
301,-1.898540,-1.80797,2.62168,3184.46,0,xy,0001,301
302,0.648398,-1.29577,1.44894,3665.35,0,xy,0001,302
303,2.403990,1.39632,2.78008,3281.94,0,xy,0001,303
304,-1.838880,-2.84160,3.38470,3256.92,0,xy,0001,304


# 3. Combine dataframes 

In [13]:
%%time
iter = 0
for each in lines[1:]:
    fname ="./data/"+each.split('.csv')[0]+".csv"
#    print(fname)
    cluster_id = each.split('_')[1]
    proj_ax = each.split('.')[0].split('_')[2]

#    print(fname)
    rawdf = pd.read_csv(fname,skiprows=1)
#    print(len(rawdf.index))

    rawdf['proj_axis'] = proj_ax
    rawdf['gal_id'] = rawdf.index
    rawdf['cluster_id'] = cluster_id

    fdf = fdf.append(rawdf, ignore_index=True)
    iter = iter +1
    if iter % 100 == 0:
        print(iter)
#    if i == 0: df = rawdf.copy() 
#    if i > 0 : df = df.append(rawdf)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
CPU times: user 6min 43s, sys: 3min 17s, total: 10min
Wall time: 10min 6s


In [14]:
fdf.count()

x             3207160
y             3207160
R_proj        3207160
v_los         3207160
flag          3207160
proj_axis     3207160
cluster_id    3207160
gal_id        3207160
dtype: int64

In [15]:
fdf.tail()

,x,y,R_proj,v_los,flag,proj_axis,cluster_id,gal_id
3207155,-2.366860,0.946819,2.54922,233.506,0,yz,2621,408
3207156,-2.756740,1.714310,3.24630,891.324,0,yz,2621,409
3207157,-4.548310,-1.665560,4.84368,217.310,3,yz,2621,410
3207158,-2.094290,1.472960,2.56040,1645.640,0,yz,2621,411
3207159,-0.427168,-1.851130,1.89977,1639.350,0,yz,2621,412


# 4. Save the df as a `parquet`

In [16]:
import pyarrow as pa
import pyarrow.parquet as pq

In [17]:
fdf.to_parquet('clusterdata.parquet.snappy',compression='snappy')

/Users/eunbinkim/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:383: FutureWarning: RangeIndex._start is deprecated and will be removed in a future version. Use RangeIndex.start instead
  'start': level._start,
/Users/eunbinkim/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:384: FutureWarning: RangeIndex._stop is deprecated and will be removed in a future version. Use RangeIndex.stop instead
  'stop': level._stop,
/Users/eunbinkim/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:385: FutureWarning: RangeIndex._step is deprecated and will be removed in a future version. Use RangeIndex.step instead
  'step': level._step


In [18]:
!ls

1.cluster-data-comb.ipynb              frame_pickle
2.cluster-ml-data-etl.ipynb            game
3.cluster-ml-model-test.ipynb          github-clusterml
4.cluster-ml-model-test-PRACTICE.ipynb github-datascience
Hands-on-ML-book-proc.ipynb            gugu.py
Untitled.ipynb                         hap.py
Untitled1.ipynb                        housing.csv
Untitled2.ipynb                        lotto.py
array_archive.npz                      myargv.py
array_archoive.npz                     paging.py
cluster-mldata.plk                     some_array.npy
clusterdata.parquet.snappy             test.py
comb.ipynb                             test.txt
data                                   test2.txt
data.tar                               test3.csv
datacomb.py                            test4.csv
dst.txt                                tseries.csv
example.parquet.snappy
